In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [2]:
data = pd.read_csv(r'C:\Users\Admin\Desktop\Учеба\Семинары Фили\week11_googleMaps\cian_data_v2.csv')

In [3]:
text = data['описание']
y = data['цена']

In [4]:
from sklearn.model_selection import train_test_split
T_train, T_test, y_train, y_test = train_test_split(text, y, test_size = 0.2)

In [5]:
import string
import re

from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()

from nltk.corpus import stopwords
stopwords_ru = stopwords.words('russian') 

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

def prepare_text(text):
    tokens = word_tokenize(text.lower())
    clean_tokens = [m.normal_forms(word)[0]  for word in tokens if (word not in stopwords_ru) and 
                         (len(re.sub('[' + string.punctuation + ']', '', word)) > 1)]
    return ' '.join(clean_tokens)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df=10, max_df=0.8)#, max_features=4000)
T_train_v = vect.fit_transform(T_train)
T_test_v = vect.transform(T_test)

In [7]:
T_train_v.todense().shape

(28087, 12704)

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

pipe = Pipeline([('preprocessing', None), ('regressor', ElasticNet())])

param_grid = [
    {'regressor': [ElasticNet()],
     'regressor__alpha': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1],
     'regressor__l1_ratio': [0, 0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]},
    {'regressor': [RandomForestRegressor(n_estimators = 300)],
     'regressor__max_depth': [700, 1000, 2000, None]},
    {'regressor': [KNeighborsRegressor()],
     'regressor__n_neighbors': [1, 2, 3, 4, 5, 40]},
    {'regressor': [SVR()],
     'regressor__tol': [10000],
     'regressor__C': [0.1, 0.5, 1, 5, 10],
     'regressor__max_iter': [10000]}
]

cv = KFold(n_splits = 5)

grid = GridSearchCV(pipe, param_grid = param_grid, cv=cv, n_jobs = -1, return_train_score=True)

In [ ]:
grid.fit(T_train_v, y_train)

print("Средняя правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid.score(T_test, y_test)))
gridresults = pd.DataFrame(grid.cv_results_)
display(gridresults.sort_values(["rank_test_score"]).T)
best_model = grid.best_estimator_

import winsound

#Пищалка по окончании обучения
frequency = 750  # Set Frequency To 750 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, grid.predict(T_test))